# 크롤링 모델_2022.07.25
    1. 검색을 위한 url: ?search ='검색어'
    2. 검색결과 활용: 내부링크 , 외부링크 검색
    3. 해당 페이지로 이동
    4. h1 정보,  body 정보를 크롤링


In [ ]:
# brookings.edu 웹사이트, 기사 콘텐트 추출

import requests
from bs4 import BeautifulSoup

# 객체 생성을 위해
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

# url -> 내용 불러오는 함수        
def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')

# 각 요소 불러오는 함수
def scrapeBrooking(url):
    bs = getPage(url)
    title = bs.find('h1').text
    body = bs.find('div', {'class' :'post-body'}).text
    return Content(url, title, body)

url = 'https://www.brookings.edu/blog/future-development/2018/01/26/delivering-inclusive-urban-access-3-uncomfortable-truths/'

content = scrapeBrooking(url)
print('Title" {}'.format(content.title))
print('URL:{}\n'.format(content.url))
print(content.body)    

In [ ]:
# 예제1

class	Content:
    def	__init__(self,	topic,	url,	title,	body):
        self.topic =	topic
        self.url =	url
        self.title =	title
        self.body =	body
    def	print(self):
        print('New	article	found	for	topic:	{}'.format(self.topic))
        print('URL:	{}'.format(self.url))
        print('TITLE:	{}'.format(self.title))
        print('BODY:\n {}'.format(self.body))
class	Website:
    def	__init__(self,	name,	url,	searchUrl,	resultListing,	resultUrl,
                 absoluteUrl,	titleTag,	bodyTag):
        self.name =	name
        self.url =	url
        self.searchUrl =	searchUrl #	URL에 검색어 추가
        self.resultListing =	resultListing #	각 결과에 대한 정보 저장
        self.resultUrl =	resultUrl #	결과에서 정확한 URL을 추출할 때 사용
        self.absoluteUrl =	absoluteUrl #	절대 경로인지,	상대 경로인지 구분
        self.titleTag =	titleTag
        self.bodyTag =	bodyTag

        
import	requests
from	bs4	import	BeautifulSoup
class Crawler:
    def	getPage(self,	url):
        try:
            req	=	requests.get(url)
        except	requests.exceptions.RequestException:
            return	None
        return	BeautifulSoup(req.text,	'html.parser')
    def	safeGet(self,	pageObj,	selector):
        childObj =	pageObj.select(selector)
        if	childObj is	not	None	and	len(childObj)	>	0:
            return	childObj[0].get_text()
        else:
            return ' '
    def	getAllBody(self,	pageObj,	selector):
    # 해당 tag를 가지는 모든 내용을 출력함
        childObj =	pageObj.select(selector)
        bodyText =	""
        if	childObj is	not	None:
            for	i in	range	(len(childObj)):
                bodyText =	bodyText +	childObj[i].get_text()	+	'\n'
            return	bodyText
        else:
            return	''        

    def	search(self,	topic,	site):
        #	site:	Website	객체
        print('searchUrl+topic:',	site.searchUrl +	topic)	
        
        bs	=	self.getPage(site.searchUrl +	topic)
        searchResults =	bs.select(site.resultListing)
        
        for	result	in	searchResults:
            url =	result.select(site.resultUrl)[0].attrs['href']
            if(site.absoluteUrl):
                bs	=	self.getPage(url)
            else:
                bs	=	self.getPage(site.url +	url)
            if	bs	is	None:
                print('Something	was	wrong	with	that	page	or	URL.	Skipping')
                return
            title	=	self.safeGet(bs,	site.titleTag)
            #body	=	self.safeGet(bs,	site.bodyTag)	#	첫 번째 paragraph만 출력
            body	=	self.getAllBody(bs,	site.bodyTag) # 전체 기사 출력
        
            if	title	!=	''	and	body	!=	'':
                content	=	Content(topic,	url,	title,	body)
                content.print()


crawler	=	Crawler()

siteData1	=	[
['Reuters',																													#	Website.name
'http://reuters.com',																		#	Website.url
'http://www.reuters.com/search/news?blob=',	#	Website.searchUrl:	검색을 위한 URL
'div.search-result-content',		#	Website.resultListing:	검색 결과에 대한 정보
'h3.search-result-title	>	a',	# Website.resultUrl:	결과에서 URL을 추출할 때 사용할 태그
False,																								 # Website.absoluteUrl 사용 여부
'h1',																									 #	Website.titleTag
'p.Paragraph-paragraph-2Bgue']		#	Website.bodyTag
]

sites	=	[]
for	row	in	siteData1:
    sites.append(Website(row[0],	row[1],	row[2],	row[3],
                        row[4],	row[5],	row[6],	row[7]))
topics	=	['python']
for	topic	in	topics:
    print('GETTING	INFO	ABOUT:	'	+	topic)
    for	targetSite in	sites:
        crawler.search(topic,targetSite)    

In [ ]:
# <a> 태그 중 특정 속성값 가져오기 : data-testid 속성 값이 'Heading','Link'인 url 링크 검색
import requests
from bs4 import BeautifulSoup

url = 'https://www.reuters.com'
link_list = []
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

data_testid_links = soup.find_all('a', 
                            attrs={'data-testid' : ['Heading', 'Link']})

i = 0
for link in data_testid_links:
    if link['href'] not in link_list:
        link_list.append(link['href'])
        print('[{0:4}]: {1}'.format(i, link['href']))
        i += 1
        
print('link_list 길이:', len(link_list))

In [ ]:
# 동적 웹페이지 크롤링 : storePop2(1)을 실행 했을때 검색 결과 출력
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome('C:\MINSU\chromedriver')
driver.get('https://www.coffeebeankorea.com/store/store.asp')

driver.execute_script('storePop2(1)') #자바스크립트를 실행하는 코드

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#  print(soup.prettify)  # html 예쁘게 출력하는 코드


store_names =soup.select('div.store_txt > p.name > span')
store_name_list = []
for name in store_names:
    store_name_list.append(name.get_text())
    
print('매장 개수:', len(store_name_list))
print(store_name_list) # 정상 동작

store_addresses = soup.select('p.address > span')
store_address_list = [] 

for addr in store_addresses:
    print(addr.get_text())   


In [ ]:
# 동적 웹페이지 크롤링 : 자바스크립트 실행, 지점 정보 가져오기
from bs4 import BeautifulSoup
from selenium import webdriver
import time

driver = webdriver.Chrome('C:\MINSU\chromedriver')

# 자바스크립트 실행
for i in range(1, 380):
    driver.get('https://www.coffeebeankorea.com/store/store.asp')
    time.sleep(1)
    
    driver.execute_script('storePop2(%d)' %i)
    time.sleep(1)
    

soup = BeautifulSoup(html, 'html.pareser')

store_name = soup.select_one('div.store_txt > h2').text
print('매장이름: ', store_name)
store_info = soup.select('div.store_txt > table.store_table > tbody > tr > td')

store_address_list = list(store_info[2])

store_Addr = store_address_list[0]
store_phone = store_info[3].text

In [1]:
# 동적 웹페이지 크롤링 실습 예제:
from	bs4	import	BeautifulSoup
from	selenium	import	webdriver
import	pandas	as	pd
import	time
def	coffeebean_store(store_list):
    coffeebean_url =	'https://www.coffeebeankorea.com/store/store.asp'
    driver	=	webdriver.Chrome('C:\MINSU\chromedriver')

    for	i in	range(1,	380):
        driver.get(coffeebean_url)
        time.sleep(1)	#	웹페이지를 연결할 동안 1초 대기

        driver.execute_script('storePop2(%d)'	%	i)
        time.sleep(1)
        try:
            html	=	driver.page_source
            soup	=	BeautifulSoup(html,	'html.parser')
            store_name =	soup.select_one('div.store_txt >	h2').text	#	매장 이름
            store_info =	soup.select("div.store_txt >	table.store_table >	tbody >	tr	>	td")
            store_address_list =	list(store_info[2])
            store_addr =	store_address_list[0]	#	매장 주소
            store_phone =	store_info[3].text	#	매장 전화번호
            print('{}	{}	{}'.format(i+1,	store_name,	store_addr,	store_phone))
            store_list.append([store_name,	store_addr,	store_phone])
        except:
            continue

def	main():
    store_info =	[]
    coffeebean_store(store_info)
    #	DataFrame으로 변경
    coffeebean_table =	pd.DataFrame(store_info,	columns=('매장이름',	'주소',	'전화번호'))
    print(coffeebean_table.head())
    #	DataFrame을 csv파일로 저장 (utf-8로 인코딩)
    coffeebean_table.to_csv('coffeebean_branches.csv',	encoding='utf-8',	mode='w',	
index=True)
    
main()        

2	학동역 DT점	서울시 강남구 학동로 211 1층  
4	차병원점	서울시 강남구 논현로 566 강남차병원1층  
7	강남대로점	서울시 서초구 강남대로 369 1층  
12	강남에스점	서울시 서초구 서초 1306-3호  
13	청담에스점	서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호  
14	신사점	서울시 강남구 도산대로 126  
15	압구정역점	서울시 강남구 논현로 842 압구정빌딩1층  
16	역삼점	서울시 강남구 논현로 512  지상1,2층  
17	양재스포타임점	서울시 서초구 강남대로 213 24호 지하1층  
18	청담성당점	서울시 강남구 삼성로 716 LEE76빌딩2층  
19	영동점	서울 서초구 반포동 736-17 P빌딩 2층  
20	도곡점	서울시 강남구 언주로 30길 10,112 현대비젼21 112호  
21	영동고앞점	서울시 강남구 선릉로 749 1,2층  
26	압구정시티점	서울시 강남구 도산대로49길 13 1층 17,18호  
27	압구정로데오점	서울시 강남구 선릉로 157길 12 석전빌딩 1~4층  
28	서초우성점	서울시 서초구 강남대로 51길 1 511 TOWER  1층  
29	논현팍스타워점	서울시 강남구 논현 231-13호 팍스타워지하1층  
30	삼성오크우드점	서울 강남구 영동대로 513 컨벤션별관A동 지하 2층  
31	트레이드타워점	서울시 강남구 영동대로 511 트레이드타워 지하1층  
32	삼성봉은사거리점	서울시 강남구 영동대로 607 1,2층  
35	반포엘루체점	서울시 서초구 신반포로 23 1141호 엘루체백화점내1층1013~1015호  
36	잠실신천점	서울시 송파구 석촌호수로 118 1층  
37	서초지파이브점	서울시 서초구 서초동 1685-8호 101~2호,113~4호,121호  
39	논현동수면센터점	서울시 강남구 논현로 717 1층  
42	삼성전자강남사옥점	서울시 서초구 서초대로74길 11 지하2층  
45	삼성생명강남사옥점	서울시 서초구 서초대로74길 4 삼성생명보험서초타워내지하1층  
46

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=103.0.5060.134)


In [2]:
# 교보문고 베스트셀러 크롤링 :1권

html = '''
<div class="title">
<a onclick="javascript:ecommerceClickListGA('9791162245620', 'KOR', '구글 엔지니어는 이렇게 일한다                                                                       ', '한빛미디어', '3301', '타이터스 윈터스', '컴퓨터/IT &gt; 컴퓨터공학', '카테고리_KOR_3301');" 
href="javascript:makeDetailUrl('KOR','3301','9791162245620','0 ', 'N' )">
<strong>구글 엔지니어는 이렇게 일한다</strong></a>
</div>

<div class="pub_info">
<span class="author">타이터스 윈터스</span>
<span class="publication">한빛미디어</span>
<span class="publication">	 
2022.05.10	<!-- 초판일 -->
</span>
</div>

<div class="info">
<span>지난 50년의 세월과 이 책이 입증한 사실이 한 가지 있습니다. 바로 '소프트웨어 엔지니어링의 발전은 결코 정체되지 않는다'라는 것입니다. 빠른 기술 변화 속에서 소프트웨어 엔지니어링의 중요성이 더욱</span>
</div>
'''
from bs4 import BeautifulSoup
book = BeautifulSoup(html, 'html.parser')

book_title = book.find('div', attrs={'class': 'title'}).find('strong').text

book_info = book.find('div', attrs={'class':'pub_info'})
book_author = book_info.find('span', attrs={'class': 'author'}).text
book_publication = book_info.find('span', attrs={'class': 'publication'}).text
book_summary = book.find('div', attrs={'class': 'info'}).find('span').text

print('title:', book_title)
print('author:', book_author)
print('publication:', book_publication)
print('summary:', book_summary)

title: 구글 엔지니어는 이렇게 일한다
author: 타이터스 윈터스
publication: 한빛미디어
summary: 지난 50년의 세월과 이 책이 입증한 사실이 한 가지 있습니다. 바로 '소프트웨어 엔지니어링의 발전은 결코 정체되지 않는다'라는 것입니다. 빠른 기술 변화 속에서 소프트웨어 엔지니어링의 중요성이 더욱


In [11]:
# 교보문고 베스트셀러 크롤링 : 여러권

import	time
from	bs4	import	BeautifulSoup
import	pandas	as	pd
from	itertools import	count
from	selenium	import	webdriver
'''
category_num1	=	[3301,	3302,	3303,	3305,	3307,	3309,	3310,	3311,	3312,	3313,	
3314,	3315,	3316,	3317,	3319,	3321,	3323,	3325,	3328,	3329]
category_dic1	=	{3301:	'컴퓨터공학',	3302:	'IT일반',	3303:	'컴퓨터입문_활용’,	
3305:	'전산통계_해석',	3307:	'OS',	3309:	'네트워크',	3310:	'보안_해킹’,	
3311:	'데이터베이스',	3312:	'개발방법론',	3313:	'게임’,	
3314:	'웹프로그래밍',	3315:	'프로그래밍언어',	3316:	'모바일프로그래밍’,	
3317:	'OA_사무자동화',	3319:	'웹사이트',	3321:	'그래픽',	3323:	'멀티미디어’,	
3325:	'CAD',	3328:	'자격증_수험서',	3329:	'대학교재'}
'''

# 카테고리 넘버 정리 , 넘버 딕셔너리 생성.
category_num =	[3301,	3311,	3312,	3315,	3316]
category_dic =	{3301:	'컴퓨터공학',	3311:	'데이터베이스',	3312:	'웹프로그래밍',	3315:	
'프로그래밍언어',	3316:	'모바일프로그래밍'}

def	get_kobomungo_data():
    wd	=	webdriver.Chrome('C:\MINSU\chromedriver')		#	본인의 webdriver 경로

    for	category_idx in	category_num:
        kobomungo_URL =	"http://www.kyobobook.co.kr/categoryRenewal/categoryMain.laf?linkClass=%s&mallGb=KOR&orderClick=sgx"	%str(category_idx)

        wd.get(kobomungo_URL)
        print("Category	Index	{}:	{}".format(category_idx,	category_dic.get(category_idx)))
        time.sleep(5)
        bestseller_list =	[]

        for	page_idx in	count():
            try:
                wd.execute_script("_go_targetPage('%s')"	%str(page_idx +	1)) # count 함수 start default=0 이므로 +1 해준다.
                print("Page	index	[%s]"	%(str(page_idx +	1)))
            except	Exception	as	e: # 더 읽을 페이지 없으면 종료
                break
            time.sleep(3)
        
            html	=	wd.page_source # page_source: 드라이버가 위치한 웹페이지의 소스코드를 얻을 수 있는 명령어
            soupData =	BeautifulSoup(html,	'html.parser')
            book_list =	soupData.find_all('li',	attrs={'class':	'id_detailli'}) # 1권 단위의 속성이 모두 담겨있는 태크,클래스를 통해 전체 책 정보 리스트 만듬.
        
            for	book	in	book_list:  # 북리스트에서 각각 필요한 세부정보 저장.
        #	<em class="best_flag"><span>1</span></em>
                book_rank =	book.find('em',	attrs={'class':	'best_flag'}).find('span').text
                book_title =	book.find('div',	attrs={'class':	'title'}).find('strong').text
                book_info =	book.find('div',	attrs={'class':	'pub_info'})
                book_author =	book_info.find('span',	attrs={'class':	'author'}).text
                book_publication =	book_info.find('span',	attrs={'class':	'publication'}).text
                book_summary =	book.find('div',	attrs={'class':	'info'}).find('span').text
                
                print(book_rank,	book_title)
                
                bestseller_list.append([book_rank,	book_title,	book_author, book_publication,	book_summary])
            
            #	파일 저장
            category_name =	category_dic.get(category_idx)	#	category[0]:	분류번호
            book_table =	pd.DataFrame(bestseller_list, columns=('순위',	'제목',	'저자',	'출판사',	'요약'))
            book_table.to_csv("./%s분야_베스트셀러_list.csv" %str(category_name), encoding="utf-8",	mode='w',	index=False)
        
def	main():
    print('교보문고 베스트셀러 크롤링 시작 ')
    get_kobomungo_data()
    print('----크롤링 종료 ------')
main()

교보문고 베스트셀러 크롤링 시작 
Category	Index	3301:	컴퓨터공학
Page	index	[1]
1 구글 엔지니어는 이렇게 일한다
2 혼자 공부하는 머신러닝+딥러닝
3 클린 아키텍처: 소프트웨어 구조와 설계의 원칙
4 한 권으로 읽는 컴퓨터 구조와 프로그래밍
5 밑바닥부터 시작하는 딥러닝
6 운영체제(쉽게 배우는)(IT COOKBOOK 238)
7 딥러닝 파이토치 교과서
8 파이토치 딥러닝 마스터
9 파이토치와 유니티 ML-Agents로 배우는 강화학습(위키북스 데이터 사이언스 시리즈 82)
10 핸즈온 머신러닝(2판)
11 이펙티브 엔지니어
12 윤성우의 열혈 자료구조
13 가상 면접 사례로 배우는 대규모 시스템 설계 기초(프로그래밍인사이트)
14 데싸노트의 실전에서 통하는 머신러닝(Must Have)
15 MLOps 도입 가이드
16 알고리즘 문제 해결 전략 세트(프로그래밍 대회에서 배우는)(전2권)
17 소프트웨어 테스팅 실무(개발자도 알아야할)(3판)
18 소프트웨어 테스트 전문가(CSTS) 가이드(개정판)
19 인공지능 바이블
20 밑바닥부터 시작하는 딥러닝 2
Page	index	[2]
21 밑바닥부터 시작하는 딥러닝 3
22 적정 소프트웨어 아키텍처
23 Do it! BERT와 GPT로 배우는 자연어 처리
24 마이크로서비스 패턴
25 코딩 테스트를 위한 자료 구조와 알고리즘 with C++
26 AI 상식사전
27 텐서플로 2와 머신러닝으로 시작하는 자연어 처리(2판)(위키북스 데이터 사이언스 시리즈 79)
28 만들면서 배우는 파이토치 딥러닝
29 인공지능은 게임을 어떻게 움직이는가?
30 모두의 딥러닝(3판)
31 선형대수와 통계학으로 배우는 머신러닝 with 파이썬
32 수학과 함께하는 AI 기초(EBS)
33 쉽게 배우는 알고리즘(개정판)(IT Cookbook 한빛 교재 시리즈 229)
34 fastai와 파이토치가 만나 꽃피운 딥러닝
35 패턴 인식과 머신 러닝(제이펍의 인공지능 시리즈 11)
36 Apache Airflow

In [12]:
df = pd.read_csv('컴퓨터공학분야_베스트셀러_list.csv')
df

,순위,제목,저자,출판사,요약
0,1,구글 엔지니어는 이렇게 일한다,타이터스 윈터스,한빛미디어,지난 50년의 세월과 이 책이 입증한 사실이 한 가지 있습니다. 바로 '소프트웨어 ...
1,2,혼자 공부하는 머신러닝+딥러닝,박해선,한빛미디어,"이 책은 수식과 이론으로 중무장한 머신러닝, 딥러닝 책에 지친 ‘독학하는 입문자’가..."
2,3,클린 아키텍처: 소프트웨어 구조와 설계의 원칙,로버트 C. 마틴,인사이트,“살아있는 전설이 들려주는 실용적인 소프트웨어 아키텍처 원칙”\n\n소프트웨어 아키...
3,4,한 권으로 읽는 컴퓨터 구조와 프로그래밍,조너선 스타인하트,책만,"컴퓨터는 어떻게 동작하며, 컴퓨터 프로그래밍이란 무엇인가? 개발자가 되기 위해 프로..."
4,5,밑바닥부터 시작하는 딥러닝,사이토 고키,한빛미디어,"『밑바닥부터 시작하는 딥러닝』은 라이브러리나 프레임워크에 의존하지 않고, 딥러닝의 ..."
...,...,...,...,...,...
295,296,New AI와 컴퓨팅 사고력,김경록,에이치북스,컴퓨팅 사고력은 컴퓨터과학의 기본적인 개념과 원리를 기반으로 효율적으로 문제를 해결...
296,297,인공지능 개론(IT Cookbook 한빛 교재 시리즈 110),마이클 네그네빗스키,한빛아카데미,『인공지능 개론』은 인공지능에 대한 지적 호기심을 충족시킬 수 있는 다양한 이론과 ...
297,298,정보시스템감리 운영 및 유지보수 감리 점검 가이드(Ver.2.0),한국정보화진흥원,진한엠앤비,행정기관 등에서 추진하는 정보시스템 운영·유지보수 사업의 감리 점검을 위한 세부적인...
298,299,코딩없는 개발 DevOn NCD,김종윤,박영사,본서에서 소개하는 “DevOn NCD(No Coding Development)”는 ...
